In [1]:
import kagglehub

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

100%|██████████| 2.29G/2.29G [00:11<00:00, 223MB/s]

Extracting files...


Path to dataset files: /teamspace/studios/this_studio/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2


In [2]:
!cp -r /teamspace/studios/this_studio/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2 ./

In [25]:
import h5py
import numpy as np
import random
import os
import cv2

TRAIN_PATH = "/teamspace/studios/this_studio/2/chest_xray/train"
NORMAL_TARGET = 900
PNEUMONIA_TARGET = 3600
IMG_SIZE = 224
OUTFILE = "train_pneumonia_subset_correct.h5"

normal_dir = os.path.join(TRAIN_PATH, "NORMAL")
pneumonia_dir = os.path.join(TRAIN_PATH, "PNEUMONIA")

normal_images = os.listdir(normal_dir)
pneumonia_images = os.listdir(pneumonia_dir)

random.seed(42)
random.shuffle(normal_images)
random.shuffle(pneumonia_images)

normal_images = normal_images[:NORMAL_TARGET]
pneumonia_images = pneumonia_images[:PNEUMONIA_TARGET]

# Load images
def load_image_safe(path, size):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv2.resize(img, (size, size))
    return img.astype("uint8")

X, y = [], []

for img_name in normal_images:
    img = load_image_safe(os.path.join(normal_dir, img_name), IMG_SIZE)
    if img is not None:
        X.append(img)
        y.append(0)

for img_name in pneumonia_images:
    img = load_image_safe(os.path.join(pneumonia_dir, img_name), IMG_SIZE)
    if img is not None:
        X.append(img)
        y.append(1)

X = np.array(X, dtype="uint8").reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y, dtype="uint8")

# Save
with h5py.File(OUTFILE, "w") as h5:
    h5.create_dataset("images", data=X, compression="gzip")
    h5.create_dataset("labels", data=y, compression="gzip")

print("Saved corrected H5:", OUTFILE)
print("Class counts:", np.bincount(y))


Saved corrected H5: train_pneumonia_subset_correct.h5
Class counts: [ 900 3600]
